# Installations

In [2]:
!pip install -q -U transformers accelerate bitsandbytes

In [3]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2701, done.
remote: Counting objects: 100% (1204/1204), done.
remote: Compressing objects: 100% (370/370), done.
remote: Total 2701 (delta 939), reused 898 (delta 834), pack-reused 1497
Receiving objects: 100% (2701/2701), 2.04 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (1698/1698), done.


In [4]:
!pip install -U pip
!pip install -e ColBERT/['faiss-gpu','torch']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///kaggle/working/ColBERT
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torc

In [5]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [6]:
!pip install neural-cherche

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for neural-cherche: filename=neural_cherche-1.1.0-py3-none-any.whl size=40021 sha256=d251d5b580b75f1f8a9c9c68803f5003191666dcc5e86e16710376e97bee2a9d
  Stored in directory: /root/.cache/pip/wheels/8b/3d/e7/c18f5627e0fd6be4d0e2792a4c8a4aa63ba19227d568de5ad2
Successfully built neural-cherche


# ColBERT

In [17]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Trainer

In [18]:
import pandas as pd
from transformers import AutoTokenizer
import random
import json

In [19]:
df = pd.read_json('/kaggle/input/rumor-verification/English_train_processed.json', lines=True)

In [20]:
df.head()

,id,rumor,label,timeline,evidence
0,AuRED_014,“#Urgent / Ramallah Ministry of Health spokesm...,REFUTES,"[[https://twitter.com/ibrahimmilhim, 135727045...","[[https://twitter.com/palestine_moh, 135721171..."
1,AuRED_037,Macron to Sky News: After my visit to Mrs. Fai...,REFUTES,"[[https://twitter.com/skynewsarabia, 130202992...","[[https://twitter.com/salmanonline, 1301079467..."
2,AuRED_085,Saudi Arabia evacuated 10 students from China ...,REFUTES,"[[https://twitter.com/YSUCORG, 122648204300706...","[[https://twitter.com/YSUCORG, 122648204300706..."
3,AuRED_089,"The Corona epidemic has reached the Emirates, ...",REFUTES,"[[https://twitter.com/WHOEMRO, 122361476919590...","[[https://twitter.com/WHOEMRO, 122250682869479..."
4,AuRED_135,The official spokesman for the Football Associ...,REFUTES,"[[https://twitter.com/AlAhlyTV, 15861380706925...","[[https://twitter.com/AlAhlyTV, 15850127731251..."


In [21]:
df_timeline = pd.read_csv('/kaggle/input/rumor-verification/timelines.csv')

# Extract strings from the second column and store them in a list
collection = df_timeline['0'].tolist()

In [22]:
rumors = []
evidences = []

for ind in df.index:
    rumors.append(df['rumor'][ind])
    evidences.append(df['evidence'][ind])

In [23]:
# Creating a TSV file for training documents (collection)
with open("collections.tsv", "w", encoding="utf-8") as collection_file:
    for i, item in enumerate(collection):
        collection_file.write(f"{i}\t{item}\n")

# Creating a TSV file for rumors
with open("queries.tsv", "w", encoding="utf-8") as queries_file:
    for i, query in enumerate(rumors):
        queries_file.write(f"{i}\t{query}\n")

with open("triples.jsonl", "w", encoding="utf-8") as triples_file:
    for i, query in enumerate(rumors):
        query_id = i

        # Extracting relevant document indices from the 'i'th index of evidences
        # print(i)
        rel_doc_indices = [collection.index(doc_id[2]) for doc_id in evidences[i]]

        evidences_ids = set(e[2] for e in evidences[i])

        # Creating triples for each relevant document
        for rel_doc_index in rel_doc_indices:
            # Searching for a non-relevant document index not present in evidences[i]
            # irrel_doc_index = random.choice([idx for idx, doc_id in enumerate(collection) if len(evidences[i]) == 3 and doc_id not in evidences[i][2]])
            irrel_doc_index = random.choice([idx for idx, doc_id in enumerate(collection) if doc_id not in evidences_ids])

            # Writing the triple to the triples.jsonl file
            triples_file.write(json.dumps([query_id, rel_doc_index, irrel_doc_index]) + '\n')

In [24]:
triples_data = []
with open("triples.jsonl", "r", encoding="utf-8") as triples_file:
    for line in triples_file:
        triple = json.loads(line.strip())
        triples_data.append(triple)
# Create a dictionary to map document indices to document text
document_index_to_text = {i: text for i, text in enumerate(collection)}

# print(triples_data[0:5])

# Retrieve information in the form similar to X
retrieved_info = [
    (df.iloc[query_id]['rumor'], document_index_to_text[rel_doc_index], document_index_to_text[irrel_doc_id])
    for query_id, rel_doc_index, irrel_doc_id in triples_data
]

# Print the retrieved information
for info in retrieved_info:
    print(info)

('“#Urgent / Ramallah Ministry of Health spokesman Kamal Al-Shakhra: We received 2,000 doses of the American “Moderna” #Corona vaccine, and this batch will be designated for President Abbas, the Fatah Central Committee, and VIPs.”', 'Pictures from the launch of the vaccination campaign against the #Coronavirus, starting with the medical and health teams in #Bethlehem. #vaccine #Palestine #COVID19 https://t.co/vnhGmoZtgC', 'Happy birthday to former Tunisian international player Jamal Sayhi!  Sending your well wishes to Jamal @jamelsaihi  https://t.co/ROudgAK0uF')
('“#Urgent / Ramallah Ministry of Health spokesman Kamal Al-Shakhra: We received 2,000 doses of the American “Moderna” #Corona vaccine, and this batch will be designated for President Abbas, the Fatah Central Committee, and VIPs.”', 'The Ministry added that the second group that will receive the vaccine is the elderly over the age of 60 and those with chronic diseases, because if they are infected with the virus, they will be m

In [25]:
print(len(retrieved_info))
X = retrieved_info

276


In [26]:
from neural_cherche import models, utils, train
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16
epochs = 5

model = models.ColBERT(
    model_name_or_path="sentence-transformers/all-mpnet-base-v1",
    device=device
)

# model = models.ColBERT(
#     model_name_or_path="checkpoint",
#     device=device
# )

# Put the training code here
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

for anchor, positive, negative in utils.iter(
        X,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=False
    ):

    loss = train.train_colbert(
        model=model,
        optimizer=optimizer,
        anchor=anchor,
        positive=positive,
        negative=negative,
    )

model.save_pretrained("checkpoint")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v1 were not used when initializing MPNetForMaskedLM: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForMaskedLM were not initialized from the model check

ColBERT(
  (model): MPNetForMaskedLM(
    (mpnet): MPNetModel(
      (embeddings): MPNetEmbeddings(
        (word_embeddings): Embedding(30527, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): MPNetEncoder(
        (layer): ModuleList(
          (0): MPNetLayer(
            (attention): MPNetAttention(
              (attn): MPNetSelfAttention(
                (q): Linear(in_features=768, out_features=768, bias=True)
                (k): Linear(in_features=768, out_features=768, bias=True)
                (v): Linear(in_features=768, out_features=768, bias=True)
                (o): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (dro

In [17]:
# from neural_cherche import models
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"

# model = models.ColBERT(
#     model_name_or_path="checkpoint",
#     device=device
# )

In [32]:
# Modify the existing collection to include the index
modified_collection = [
    {"id": i, "text": text}
    for i, text in enumerate(collection)
]

# Print the modified collection
print(modified_collection[0:5])

[{'id': 0, 'text': '“Motherhood and childhood have the right to special care and assistance. All children have the right to the same social protection whether born in or out of marriage.” - Article 25 of the Universal Declaration of Human Rights #StandUp4HumanRights https://t.co/DgfKdgGF0Q'}, {'id': 1, 'text': '#Health during the daily press conference on the #Coronavirus: The number of countries in which the virus appeared reached 163 countries...and the total number of infected people around the world is 200 thousand cases'}, {'id': 2, 'text': '"Breaking | Regional spokesman for the "#US State Department": #Washington is ready to play a positive role in the #Renaissance_Dam negotiations and we will not link American aid to Ethiopia to the file https://t.co/6n9jTUEycR"'}, {'id': 3, 'text': '- Akkar Disaster Management: No new injuries https://t.co/63sCT6fvWT'}, {'id': 4, 'text': 'formation | Formation of the two teams for the #FCPAHL #TotalCAFCL match https://t.co/63zh3tmr6i'}]


In [36]:
# third_elements = [row[2] for row in df.iloc[0]['timeline']]
# print(len(third_elements))
# print(third_elements)
# rumors[0]
# rumors

75
['"Qalqilya (5), Bethlehem (10), Nablus (24), Jericho and the Jordan Valley (9), Ramallah and Al-Bireh (0), Tulkarm (28), Hebron (23), Jenin (19)", Gaza Strip ( 275). The Ministry of Health noted that there were 55 patients in intensive care rooms, including 20 patients on ventilators.', 'Salfit (49), Jerusalem suburbs (61), Tubas (18), Qalqilya (12), Bethlehem (42), Nablus (53), Jericho and Al-Aghwar (13), Ramallah and Al-Bireh (127), Tulkarm (18) Hebron (44), Jenin (24), Gaza Strip (200). She added that the new recoveries were distributed as follows: "Salfit (10), Jerusalem suburbs (0), Tubas (16)."', '“While the percentage of active infections reached 4.9%, and the death rate was 1.1% of the total infections. 7 deaths were recorded in the West Bank: “Jericho and the Jordan Valley 1, Bethlehem 1, Jenin 1, Salfit 1, Jerusalem Suburbs 1, Nablus 2.” Gaza Strip, one death. She indicated that the new infections were recorded as follows: “', 'Epidemiological report on the Corona virus i

In [34]:
from neural_cherche import models, rank, retrieve
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32

documents = modified_collection

retriever = retrieve.TfIdf(
    key="id",
    on=["text"],
)

model = models.ColBERT(
    model_name_or_path="/kaggle/working/checkpoint",
    device=device,
)

ranker = rank.ColBERT(
    key="id",
    on=["text"],
    model=model
)

retriever_documents_embeddings = retriever.encode_documents(
    documents=documents,
)

retriever.add(
    documents_embeddings=retriever_documents_embeddings,
)

ranker_documents_embeddings = ranker.encode_documents(
    documents=documents,
    batch_size=batch_size,
)

retriever_queries_embeddings = retriever.encode_queries(
    queries=rumors,
)

ranker_queries_embeddings = ranker.encode_queries(
    queries=rumors,
    batch_size=batch_size,
)

candidates = retriever(
    queries_embeddings=retriever_queries_embeddings,
    k=1000,
)

scores = ranker(
    documents=candidates,
    queries_embeddings=ranker_queries_embeddings,
    documents_embeddings=ranker_documents_embeddings,
    k=100,
    batch_size=32,
)

100%|██████████| 32/32 [00:00<00:00, 432.81it/s]


In [100]:
timeline = []
tmp_timeline = []

for ind, inner_list in enumerate(scores):
    print(ind)
    # Sort the inner list by similarity in descending order
    sorted_inner_list = sorted(inner_list, key=lambda x: x['similarity'], reverse=True)

    # Extract the top 10 elements
    top_10_elements = sorted_inner_list[:10]

    # Print the corresponding elements from the collection using the 'id'
    predicted_document = []
    evidences_document = set(e[2] for e in evidences[ind])
    ind_score = 0
    tot_score = len(evidences_document)

    for element in top_10_elements:
        element_id = element['id']
        corresponding_element = modified_collection[element_id]

#         print(corresponding_element['text'])
        tmp_timeline.append(corresponding_element['text'])
        if corresponding_element['text'] in evidences_document:
            ind_score += 1
    if tot_score == 0:
        print("Percent score: 0" + "   Score: " + str(ind_score) + "   Tot_score: " + str(tot_score))
    else:
        print("Percent score: " + str(ind_score/tot_score) + "   Score: " + str(ind_score) + "   Tot_score: " + str(tot_score))
    timeline.append(tmp_timeline)
    tmp_timeline = []
    print()

0
Percent score: 0.5   Score: 2   Tot_score: 4

1
Percent score: 0.3333333333333333   Score: 1   Tot_score: 3

2
Percent score: 0.3333333333333333   Score: 10   Tot_score: 30

3
Percent score: 0.2   Score: 2   Tot_score: 10

4
Percent score: 0.5   Score: 1   Tot_score: 2

5
Percent score: 1.0   Score: 1   Tot_score: 1

6
Percent score: 0.0   Score: 0   Tot_score: 1

7
Percent score: 1.0   Score: 1   Tot_score: 1

8
Percent score: 0.0   Score: 0   Tot_score: 1

9
Percent score: 1.0   Score: 1   Tot_score: 1

10
Percent score: 0.25   Score: 8   Tot_score: 32

11
Percent score: 1.0   Score: 1   Tot_score: 1

12
Percent score: 1.0   Score: 3   Tot_score: 3

13
Percent score: 0.8   Score: 4   Tot_score: 5

14
Percent score: 1.0   Score: 2   Tot_score: 2

15
Percent score: 0.8333333333333334   Score: 5   Tot_score: 6

16
Percent score: 1.0   Score: 2   Tot_score: 2

17
Percent score: 1.0   Score: 1   Tot_score: 1

18
Percent score: 0   Score: 0   Tot_score: 0

19
Percent score: 0   Score: 0 

In [101]:
predicted_data = {'id': df['id'],
                  'rumor': df['rumor'],
                  'label': df['label'],
                  'timeline': timeline
                 }

predicted_df = pd.DataFrame(predicted_data)

In [102]:
predicted_df.head(5)

,id,rumor,label,timeline
0,AuRED_014,“#Urgent / Ramallah Ministry of Health spokesm...,REFUTES,"[Yesterday, the Ministry of Health announced t..."
1,AuRED_037,Macron to Sky News: After my visit to Mrs. Fai...,REFUTES,[“Fairouz gives us hope and gives a dreamy pic...
2,AuRED_085,Saudi Arabia evacuated 10 students from China ...,REFUTES,[RT @Yemen_PM: Prime Minister Dr. Maeen Abdulm...
3,AuRED_089,"The Corona epidemic has reached the Emirates, ...",REFUTES,[Today @WHO confirmed the emergence of the fir...
4,AuRED_135,The official spokesman for the Football Associ...,REFUTES,[“Al-Ahly’s objection speech to the “Zamalek u...


# Mistral

## Setting the Quantization for Mistral

In [48]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True
)

## Mistral Loading

In [132]:
from transformers import AutoTokenizer, AutoModelForCausalLM 
import torch

model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Creating the Mistral Pipeline

In [133]:
from transformers import pipeline 

mistral_pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"
)

# Text-Generation

In [143]:
# Prompt Construction
def prompt_constructor(index):
    prompt = "You have to prove the following rumor is supported or refuted by the tweets given. If either of them can be done, just write either SUPPORTS or REFUTES"
    prompt = prompt + "\nIf it is not possible to prove, tell NOT ENOUGH INFO\n\n."
    prompt = prompt + "The output should be in this format. REFUTES The tweets which support or refute the rumor:\nPictures from the launch of the vaccination campaign against the #Coronavirus, starting with the medical and health teams in #Bethlehem. #vaccine #Palestine #COVID19 https://t.co/vnhGmoZtgC\nThe Ministry added that the second group that will receive the vaccine is the elderly over the age of 60 and those with chronic diseases, because if they are infected with the virus, they will be more susceptible to serious symptoms. #vaccine #modernavaccine #palestine #COVID19"
    prompt = prompt + "\nTo prove it, use the following tweets and see which tweets support or refute the rumor:\nOutput upto 5 tweets which prove or refute the rumor as it is with the number\n\n"
    prompt = prompt + str(df.iloc[index]['rumor']) + "\n\n"

    for i, tweet in enumerate(predicted_df.iloc[0]['timeline']):
        prompt = prompt + str(i) + ") " + tweet + "\n\n"

    return prompt

In [144]:
prompt = prompt_constructor(0)
print(prompt)

You have to prove the following rumor is supported or refuted by the tweets given. If either of them can be done, just write either SUPPORTS or REFUTES
If it is not possible to prove, tell NOT ENOUGH INFO

.The output should be in this format. REFUTES The tweets which support or refute the rumor:
Pictures from the launch of the vaccination campaign against the #Coronavirus, starting with the medical and health teams in #Bethlehem. #vaccine #Palestine #COVID19 https://t.co/vnhGmoZtgC
The Ministry added that the second group that will receive the vaccine is the elderly over the age of 60 and those with chronic diseases, because if they are infected with the virus, they will be more susceptible to serious symptoms. #vaccine #modernavaccine #palestine #COVID19
To prove it, use the following tweets and see which tweets support or refute the rumor:
Output upto 5 tweets which prove or refute the rumor as it is with the number

“#Urgent / Ramallah Ministry of Health spokesman Kamal Al-Shakhra:

In [151]:
predicted_labels = []
predicted_evidences = []

for i in range(len(df)):
    predicted_evidence = []
    prompt = prompt_constructor(0)

    sequences = mistral_pipe(
        prompt,
        do_sample=True,
        max_new_tokens=1024,
        temperature=0.1,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )
    answer = sequences[0]['generated_text'].splitlines()
    
    predicted_labels.append(answer[0])
    for k in range(1,len(answer)):
        predicted_evidence.append(answer[k])
    
    predicted_evidences.append(predicted_evidence)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [152]:
file_output = open("output.txt", "w", encoding="utf-8")
for evidence in predicted_evidences:
    for j in range(len(evidence)):
        file_output.write(evidence[j])
        file_output.write('\n')